In [1]:
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import copy
import multiprocessing as mp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score, log_loss
from tqdm import tqdm
from scipy import sparse, spatial
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

/root/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
import os
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = "0"
config = tf.compat.v1.ConfigProto()
# config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
# session = tf.Session(config=config)

from keras.models import Sequential, Model
from keras.layers import Input, Dense, CuDNNLSTM, Bidirectional, Embedding, CuDNNGRU, Conv1D, MaxPooling1D, AveragePooling1D, GlobalMaxPooling1D
from keras.layers import Flatten, PReLU, Dropout, BatchNormalization, SpatialDropout1D, concatenate, Concatenate
from keras.layers import Add, Subtract, Multiply, Dot
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing.sequence import pad_sequences
from keras.utils import Sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.engine.topology import Layer 
# from keras.utils.training_utils import multi_gpu_model

Using TensorFlow backend.


In [3]:
tic = time.time()
DATA_PATH = '../pkl/'
SAVE_PATH = './feats/'
WEIGHT_PATH = './weights/'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)
if not os.path.exists(WEIGHT_PATH):
    print('create dir: %s' % WEIGHT_PATH)
    os.mkdir(WEIGHT_PATH)

print('title_desc_textcnn_enc...')

title_desc_textcnn_enc...


In [4]:
data = pd.read_pickle(os.path.join(DATA_PATH, 'invite_data.pkl'))
question_info = pd.read_pickle(os.path.join(DATA_PATH, 'question_info.pkl'))
word = pd.read_pickle(os.path.join(DATA_PATH, 'word.pkl'))
len_train = (~data['label'].isnull()).sum()
print(len_train)

9489162


In [5]:
data.head()

,iday,ihour,itime,label,qid,uid
0,3865,22,92782,0.0,2166419046,401693808
1,3844,11,92267,0.0,1550017551,3392373099
2,3862,15,92703,0.0,604029601,2317670257
3,3849,11,92387,0.0,2350061229,1618461867
4,3867,4,92812,0.0,2443223942,3544409350


In [6]:
question_info.head()

,qid,qtime,title_sw,title_w,desc_sw,desc_w,topic,qday,qhour
0,2234111670,24437,"[211, 204, 1715, 69, 2033, 138, 57, 138, 8, 28...","[22414, 963, 10458]",[0],[0],"[321, 730, 5784, 4389]",1018,5
1,760329790,41900,"[69, 2033, 138, 2616, 2668, 36, 2594, 1165, 20...","[12677, 16829, 15201, 6419, 101839]","[146, 982, 401, 297, 17, 2616, 2668, 36, 2594,...","[1296, 2118, 12677, 16829, 15201, 6419, 101839...","[278, 12673, 4677]",1745,20
2,741313548,48789,"[153, 662, 1218, 853, 325, 1056, 467, 398, 102...","[700, 2781, 3280, 81215]","[1956, 3583, 153, 34, 35, 1016, 586, 586, 716,...","[732, 24400, 48321, 39608, 20788, 219486, 1183...",[226],2032,21
3,3481466230,52455,"[22, 179, 57, 451, 594, 118, 882, 655, 1, 433,...","[3312, 1823, 1505, 638, 166, 461]","[323, 37, 1, 606, 1227, 29, 22, 179, 7, 44, 27...","[6642, 4214, 3312, 1505, 2205, 232, 294, 7177,...","[51, 4468]",2185,15
4,3966197028,54473,"[1622, 223, 1218, 853, 390, 220, 753, 909, 557...","[700, 895, 2253]",[0],[0],"[54700, 81, 57, 17670, 43574]",2269,17


In [7]:
data = data.merge(question_info[['qid', 'title_w', 'desc_w']], 'left', 'qid')
# data['topic'] = data['topic'].fillna('-1')
data.head()

,iday,ihour,itime,label,qid,uid,title_w,desc_w
0,3865,22,92782,0.0,2166419046,401693808,"[11058, 272, 2202, 431, 951, 243, 3828, 3037, ...","[3828, 14870, 3037, 2450, 692, 1688, 53547, 24..."
1,3844,11,92267,0.0,1550017551,3392373099,"[149, 79, 5210, 22869]","[429, 2709, 1465, 6965]"
2,3862,15,92703,0.0,604029601,2317670257,"[31489, 9218, 3440, 243]","[3046, 221, 3440, 730, 31489, 9218, 1278, 415]"
3,3849,11,92387,0.0,2350061229,1618461867,"[973, 64958, 8583, 2269, 628, 565, 2200, 590, ...",[0]
4,3867,4,92812,0.0,2443223942,3544409350,"[554, 28208, 396, 51]",[0]


In [8]:
max_len_t = data['title_w'].apply(len).max()
max_len_d = data['desc_w'].apply(len).max()
print(max_len_t, max_len_d) # 110

38 973


In [9]:
max_len_d = 256

In [10]:
np.random.seed(42)
def shuffle(l):
    l = l[:256]
    np.random.shuffle(l)
    return l

data['title_w'] = data['title_w'].progress_apply(shuffle)
data['desc_w'] = data['desc_w'].progress_apply(shuffle)

100%|██████████| 10630845/10630845 [00:38<00:00, 274025.74it/s]


In [11]:
data.head()

,iday,ihour,itime,label,qid,uid,title_w,desc_w
0,3865,22,92782,0.0,2166419046,401693808,"[3037, 272, 243, 11058, 263, 2202, 951, 431, 3...","[11237, 2202, 263, 33060, 1619, 1688, 272, 245..."
1,3844,11,92267,0.0,1550017551,3392373099,"[79, 149, 22869, 5210]","[2709, 1465, 429, 6965]"
2,3862,15,92703,0.0,604029601,2317670257,"[243, 9218, 31489, 3440]","[221, 3440, 415, 9218, 31489, 3046, 730, 1278]"
3,3849,11,92387,0.0,2350061229,1618461867,"[973, 2200, 2269, 64958, 628, 565, 8583, 483, ...",[0]
4,3867,4,92812,0.0,2443223942,3544409350,"[51, 396, 554, 28208]",[0]


In [12]:
title = pad_sequences(data['title_w'], maxlen=max_len_t)
desc = pad_sequences(data['desc_w'], maxlen=max_len_d)

In [13]:
title.shape

(10630845, 38)

In [14]:
desc.shape

(10630845, 256)

In [15]:
# embed_weights = np.array([[0] * 64] + [list(v) for v in word['vector'].values])
embed_weights = np.vstack([np.zeros((1, 64))] + list(word['vector'].values))

In [16]:
embed_weights.shape

(1762830, 64)

In [17]:
class DataSequence(Sequence):
    
    def __init__(self, xu, xq, y, batch_size=128):
        self.xu = xu
        self.xq = xq
        self.y = y
        self.batch_size = batch_size
         
    def __len__(self):
        return int(np.ceil(self.xu.shape[0] / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_idx = np.arange(idx * self.batch_size, min((idx + 1) * self.batch_size, self.xu.shape[0]))
        batch_xu = self.xu[batch_idx]
        batch_xq = self.xq[batch_idx]
        batch_y = self.y[batch_idx]
        return [batch_xu, batch_xq], batch_y

In [18]:
class MetricsCallback(Callback):
    def __init__(self, x_trn_u, x_trn_q, y_trn, x_val_u, x_val_q, y_val, batch_size=128, save_name='weight.h5'):
        self.trn_generator = DataSequence(x_trn_u, x_trn_q, y_trn, BATCH_SIZE)
        self.val_generator = DataSequence(x_val_u, x_val_q, y_val, BATCH_SIZE)
        self.y_trn = y_trn
        self.y_val = y_val
        self.save_name = save_name
        self.best_score = 0.5

    def on_epoch_end(self, epoch, logs={}):
        # eval train
        y_pred = self.model.predict_generator(self.trn_generator, 
                                              max_queue_size=10, 
                                              workers=1, 
                                              use_multiprocessing=False, 
                                              verbose=0)
        roc = roc_auc_score(self.y_trn, y_pred)
        # eval valid
        y_pred_val = self.model.predict_generator(self.val_generator, 
                                              max_queue_size=10, 
                                              workers=1, 
                                              use_multiprocessing=False, 
                                              verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
#         print(roc_val)
#         print(log_loss(self.y_val, y_pred_val)/len(y_pred_val))
        if roc_val > self.best_score:
            self.best_score = roc_val
            self.model.save_weights(os.path.join(WEIGHT_PATH, self.save_name))
        
        return

In [19]:
def Net():
    # title
    inp_t = Input(shape=(max_len_t,))
    x = Embedding(embed_weights.shape[0], embed_weights.shape[1], weights=[embed_weights], trainable=False)(inp_t)
    convs = []
    for kernel_size in [3, 4, 5]:
        c = Conv1D(128, kernel_size, activation='relu')(x)
        c = GlobalMaxPooling1D()(c)
        convs.append(c)
    x = Concatenate()(convs)
    # desc
    inp_d = Input(shape=(max_len_d,))
    y = Embedding(embed_weights.shape[0], embed_weights.shape[1], weights=[embed_weights], trainable=False)(inp_d)
    convs = []
    for kernel_size in [3, 4, 5]:
        c = Conv1D(128, kernel_size, activation='relu')(y)
        c = GlobalMaxPooling1D()(c)
        convs.append(c)
    y = Concatenate()(convs)
    ## dense
    x = Concatenate()([x, y])
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = PReLU()(x)
    x = Dropout(0.2)(x)
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = PReLU()(x)
    x = Dropout(0.2)(x)
    out = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=[inp_t, inp_d], outputs=out)
    return model

In [20]:
train_x_u = title[:len_train]
train_x_q = desc[:len_train]
test_x_u = title[len_train:]
test_x_q = desc[len_train:]
train_y = data['label'].values[:len_train]

In [21]:
print(train_x_u.shape)
print(test_x_u.shape)
print(train_y.sum() / len(train_y))

(9489162, 38)
(1141683, 38)
0.1773515933229931


In [23]:
BATCH_SIZE = 1024

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (tr_idx, va_idx) in enumerate(kfold.split(train_x_u, train_y)):
    
    print('-'*100)
    print('Fold %d' % i)
    trn_x_u, trn_x_q, trn_y = train_x_u[tr_idx], train_x_q[tr_idx], train_y[tr_idx]
    val_x_u, val_x_q, val_y = train_x_u[va_idx], train_x_q[va_idx], train_y[va_idx]
    
    K.clear_session()
    model = Net()
    model.compile(loss='binary_crossentropy', optimizer='adam',)
    
    trn_generator = DataSequence(trn_x_u, trn_x_q, trn_y, batch_size=BATCH_SIZE)
    val_generator = DataSequence(val_x_u, val_x_q, val_y, batch_size=BATCH_SIZE)
    
    history = model.fit_generator(generator=trn_generator, 
                        epochs=10, 
                        verbose=1, 
                        callbacks=[MetricsCallback(trn_x_u, trn_x_q, trn_y, val_x_u, val_x_q, val_y, 
                                                   batch_size=BATCH_SIZE*4, 
                                                   save_name='title_desc_textcnn_enc_weight_fold%d.h5' % i)], 
                        max_queue_size=100, 
                        workers=1, 
                        use_multiprocessing=False,)
    
    del trn_x_u, trn_x_q, trn_y, val_x_u, val_x_q, val_y
    gc.collect()

----------------------------------------------------------------------------------------------------
Fold 2
Epoch 1/10
7414/7414 [==============================] - 297s 40ms/step - loss: 0.4565
roc-auc: 0.6331 - roc-auc_val: 0.6259                                                                                                    
Epoch 2/10
7414/7414 [==============================] - 297s 40ms/step - loss: 0.4507
roc-auc: 0.649 - roc-auc_val: 0.6345                                                                                                    
Epoch 3/10
7414/7414 [==============================] - 296s 40ms/step - loss: 0.4474
roc-auc: 0.6613 - roc-auc_val: 0.6359                                                                                                    
Epoch 4/10
7414/7414 [==============================] - 296s 40ms/step - loss: 0.4446
roc-auc: 0.669 - roc-auc_val: 0.6353                                                                                                   

In [ ]:
# del trn_x_u, trn_x_q, trn_y, val_x_u, val_x_q, val_y
# gc.collect()

In [ ]:
# 0.4608

In [22]:
BATCH_SIZE = 512

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [23]:
test_generator = DataSequence(test_x_u, test_x_q, np.zeros(test_x_u.shape[0]), batch_size=BATCH_SIZE * 4)

test_pred = np.zeros((test_x_u.shape[0], 1))
train_pred = np.zeros((train_x_u.shape[0], 1))

for i, (tr_idx, va_idx) in enumerate(kfold.split(train_x_u, train_y)):
    print('-'*100)
    print('Fold %d' % i)
    trn_x_u, trn_x_q, trn_y = train_x_u[tr_idx], train_x_q[tr_idx], train_y[tr_idx]
    val_x_u, val_x_q, val_y = train_x_u[va_idx], train_x_q[va_idx], train_y[va_idx]
    
    K.clear_session()
    model = Net()
    model.compile(loss='binary_crossentropy', optimizer='adam',)
    
    trn_generator = DataSequence(trn_x_u, trn_x_q, trn_y, batch_size=BATCH_SIZE)
    val_generator = DataSequence(val_x_u, val_x_q, val_y, batch_size=BATCH_SIZE)
    
    model.load_weights(os.path.join(WEIGHT_PATH, 'title_desc_textcnn_enc_weight_fold%d.h5' % i))
    
    train_pred[va_idx] = model.predict_generator(val_generator)
    test_pred += model.predict_generator(test_generator) / 5
    
    print(roc_auc_score(val_y, train_pred[va_idx]))
    
    del trn_x_u, trn_x_q, trn_y, val_x_u, val_x_q, val_y
    gc.collect()
    print('Predict Done.')

----------------------------------------------------------------------------------------------------
Fold 0
0.6377413412107894
Predict Done.
----------------------------------------------------------------------------------------------------
Fold 1
0.6371083983113438
Predict Done.
----------------------------------------------------------------------------------------------------
Fold 2
0.6375871550969009
Predict Done.
----------------------------------------------------------------------------------------------------
Fold 3
0.6377374769203883
Predict Done.
----------------------------------------------------------------------------------------------------
Fold 4
0.638109982919724
Predict Done.


In [24]:
arr = np.vstack([train_pred, test_pred])
feat = pd.DataFrame(arr, columns=['title_desc_textcnn_enc'])
feat.head()

,title_desc_textcnn_enc
0,0.040283
1,0.213731
2,0.041021
3,0.100659
4,0.287943


In [25]:
feat.to_pickle('./feats/title_desc_textcnn_enc.pkl')

In [26]:
feat.shape

(10630845, 1)